Проверка датасета

In [ ]:
# Validate image dataset: existence, readability, and expected size
# Works with: CSV ("path,label") or TXT lines "path label" (space/comma separated)

import os, csv, sys, time
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import Counter, defaultdict
from typing import List, Tuple, Dict

from PIL import Image, ImageFile, UnidentifiedImageError
ImageFile.LOAD_TRUNCATED_IMAGES = False  # raise on truncated files

# ==== CONFIG ====
INDEX_PATH   = "/content/hirise-map-proj-v3/cleared_dataset.csv"  # or dataset.csv
ROOT_DIR     = "/content/hirise-map-proj-v3/map-proj-v3/"             # optional prefix for relative paths ("" = leave as-is)
EXPECTED_W   = 227
EXPECTED_H   = 227
MAX_WORKERS  = max(2, os.cpu_count() or 2)
LIST_EXAMPLES = 20            # how many sample bad files to print

# ==== Helpers ====

def parse_index(index_path: str) -> List[Tuple[str, str]]:
    """
    Returns list of (path, label_str).
    Supports:
      * CSV with header path,label
      * TXT lines: "path label" or "path,label"; path may contain spaces
    """
    items = []
    if index_path.lower().endswith(".csv"):
        with open(index_path, newline="") as f:
            reader = csv.DictReader(f)
            assert "path" in reader.fieldnames, "CSV must have 'path' column"
            for row in reader:
                p = row["path"].strip()
                lab = (row.get("label") or row.get("labels") or "").strip()
                items.append((p, lab))
    if ROOT_DIR:
        items = [(os.path.join(ROOT_DIR, p), lab) for p, lab in items]
    return items

def check_one(path: str) -> Dict:
    """
    Try opening the image safely. Return a dict with status and details.
    """
    info = {
        "path": path,
        "exists": os.path.exists(path),
        "ok": False,
        "err": None,
        "err_type": None,
        "size": None,
        "mode": None,
        "size_ok": None,
    }
    if not info["exists"]:
        info["err"] = "File not found"
        info["err_type"] = "NotFound"
        return info
    try:
        # First pass: verify header without decoding full image
        with Image.open(path) as im:
            im.verify()
        # Second pass: actually load to catch truncated data
        with Image.open(path) as im:
            im.load()
            w, h = im.size
            info["size"] = (w, h)
            info["mode"] = im.mode
            info["size_ok"] = (w == EXPECTED_W and h == EXPECTED_H)
        info["ok"] = True
        return info
    except UnidentifiedImageError as e:
        info["err"] = str(e)
        info["err_type"] = "UnidentifiedImageError"
    except OSError as e:
        info["err"] = str(e)
        info["err_type"] = "OSError"
    except Exception as e:
        info["err"] = str(e)
        info["err_type"] = e.__class__.__name__
    return info

# ==== Run ====
items = parse_index(INDEX_PATH)
paths = [p for p, _ in items]
print(f"Indexed items: {len(items)}")

for group_label in set([lab for _, lab in items]):
    print(f"Group {group_label}: {len([p for p, lab in items if lab == group_label])}")

t0 = time.time()
results = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futs = [ex.submit(check_one, p) for p in paths]
    for fut in as_completed(futs):
        results.append(fut.result())
dt = time.time() - t0
print(f"Checked {len(results)} files in {dt:.1f}s with {MAX_WORKERS} workers")

# ==== Summary ====
missing   = [r for r in results if not r["exists"]]
bad_read  = [r for r in results if r["exists"] and not r["ok"]]
wrong_sz  = [r for r in results if r["ok"] and r["size_ok"] is False]
ok_files  = [r for r in results if r["ok"] and r["size_ok"] is True]

modes = Counter([r["mode"] for r in results if r["mode"]])
err_types = Counter([r["err_type"] for r in bad_read])

print("\n=== SUMMARY ===")
print(f"OK (readable & {EXPECTED_W}x{EXPECTED_H}): {len(ok_files)}")
print(f"MISSING: {len(missing)}")
print(f"READ ERRORS: {len(bad_read)}  (by type: {dict(err_types)})")
print(f"WRONG SIZE: {len(wrong_sz)}")
print(f"Image modes (for OK/loaded files): {dict(modes)}")

if missing:
    print(f"\nMissing files (first {LIST_EXAMPLES}):")
    for r in missing[:LIST_EXAMPLES]:
        print("  ", r["path"])

if bad_read:
    print(f"\nUnreadable / corrupted (first {LIST_EXAMPLES}):")
    for r in bad_read[:LIST_EXAMPLES]:
        print(f"  {r['path']}  →  {r['err_type']}: {r['err']}")

if wrong_sz:
    print(f"\nWrong size (first {LIST_EXAMPLES}):")
    for r in wrong_sz[:LIST_EXAMPLES]:
        print(f"  {r['path']}  →  got {r['size']}, expected ({EXPECTED_W}, {EXPECTED_H})")


Indexed items: 21392
Group 1: 4900
Group 7: 1904
Group 3: 4662
Group 4: 3500
Group 6: 2296
Group 2: 2282
Group 5: 1848
Checked 21392 files in 13.3s with 2 workers

=== SUMMARY ===
OK (readable & 227x227): 21392
MISSING: 0
READ ERRORS: 0  (by type: {})
WRONG SIZE: 0
Image modes (for OK/loaded files): {'L': 21392}


## Аугментация

In [ ]:
import random
import numpy as np
from PIL import Image, ImageFilter
import os, csv, sys, time
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import Counter, defaultdict
from typing import List, Tuple, Dict
import tqdm

INDEX_PATH   = "/content/hirise-map-proj-v3/dataset.csv"  # or dataset.csv
ROOT_DIR     = "/content/hirise-map-proj-v3/map-proj-v3/" # optional prefix for relative paths ("" = leave as-is)
EXPECTED_W   = 227
EXPECTED_H   = 227

import random
import cv2

def add_noise(img):
    row , col = img.shape
    number_of_pixels = random.randint(300, 800)
    for i in range(number_of_pixels):

        # Pick a random y coordinate
        y_coord=random.randint(0, row - 1)

        # Pick a random x coordinate
        x_coord=random.randint(0, col - 1)

        # Color that pixel to white
        img[y_coord][x_coord] = 255

    # Randomly pick some pixels in
    # the image for coloring them black
    number_of_pixels = random.randint(300 , 800)
    for i in range(number_of_pixels):

        # Pick a random y coordinate
        y_coord=random.randint(0, row - 1)

        # Pick a random x coordinate
        x_coord=random.randint(0, col - 1)

        # Color that pixel to black
        img[y_coord][x_coord] = 0

    return img


groups_of_images = {}

with open(INDEX_PATH, "r") as f:
    for line in f.readlines()[1:]:
        parts = line.strip().split(',')
        if len(parts) < 2:
            continue
        path, label = parts[0], parts[1]
        if label not in groups_of_images:
            groups_of_images[label] = []
        groups_of_images[label].append(path)

indexed_groups_of_images = {}

for group_label in groups_of_images.keys():
    print(group_label, len(groups_of_images[group_label]))
    indexed_groups_of_images[group_label] = len(groups_of_images[group_label])

indexed_groups_of_images = sorted(indexed_groups_of_images.items(), key=lambda x: x[1])

print(indexed_groups_of_images)

print("Возьму в аугментацию группу", indexed_groups_of_images[0][0], indexed_groups_of_images[0][1])


for group_label, group_size in enumerate(indexed_groups_of_images):
    if group_label not in [2, 3, 4, 5, 6, 7]:
        continue

    for image in tqdm.tqdm(groups_of_images[str(group_label)]):
        img = cv2.imread(os.path.join(ROOT_DIR, image), cv2.IMREAD_GRAYSCALE)
        noisy_img = add_noise(img)
        new_name = os.path.join(ROOT_DIR, image.split(".")[0] + str(random.randint(1, 10000)) + "NOISE.png")
        cv2.imwrite(new_name, noisy_img)
        with open(INDEX_PATH, "a") as f:
            f.write(f"\n{new_name},{group_label}")
    print(f"#{group_label} аугментирован")


print("Аугментация завершена")

0 61054
5 924
1 4900
3 2331
2 1141
6 1148
7 952
4 1750
[('5', 924), ('7', 952), ('2', 1141), ('6', 1148), ('4', 1750), ('3', 2331), ('1', 4900), ('0', 61054)]
Возьму в аугментацию группу 5 924


100%|██████████| 1141/1141 [00:02<00:00, 410.29it/s]


#2 аугментирован


100%|██████████| 2331/2331 [00:07<00:00, 326.74it/s]


#3 аугментирован


...

#5 аугментирован


100%|██████████| 1148/1148 [00:04<00:00, 273.82it/s]


#6 аугментирован


100%|██████████| 952/952 [00:02<00:00, 370.50it/s]

#7 аугментирован
Аугментация завершена


Нейронка на базе Resnet50 и с батчам в 128
тут нужно реализовать WeightedRandomSampler и взвешенный CrossEntropyLoss


```python
from collections import Counter
counts = Counter(y_train)                     # по train
w_class = {c: 1.0/max(counts[c],1) for c in range(8)}
w_sample = [w_class[c] for c in y_train]
sampler = torch.utils.data.WeightedRandomSampler(w_sample, num_samples=len(w_sample), replacement=True)

# DataLoader(train_dataset, sampler=sampler, shuffle=False, ...)
# criterion = nn.CrossEntropyLoss(weight=torch.tensor([w_class[c] for c in range(8)], device=device), label_smoothing=0.05)

```

# Или же, убрать 0 класс
и если никакой из нейроновов не достиг больше половины, то помечать как undefined



In [ ]:
# убираем 0 класс из csv

INDEX_PATH = "/content/hirise-map-proj-v3/dataset.csv"
NEW_INDEX_PATH = "/content/hirise-map-proj-v3/cleared_dataset.csv"

items = []
with open(INDEX_PATH, newline="") as f:
    reader = csv.DictReader(f)
    for row in reader:
        p = row["path"].strip()
        lab = int(row["label"]) if "label" in row else int(row["labels"])
        items.append((p, lab))

with open(NEW_INDEX_PATH, "w") as f:
    f.write("path,label\n")
    for item in items:
        if int(item[1]) == 0:
            continue
        f.write(f"{item[0]},{item[1]}\n")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, csv, time, math, random
from typing import List, Tuple, Dict
from collections import Counter

import numpy as np
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = False

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import pandas as pd

# -----------------------
# Config
# -----------------------
INDEX_PATH   = "/content/hirise-map-proj-v3/cleared_dataset.csv"  # CSV "path,label"
ROOT_DIR     = "/content/hirise-map-proj-v3/map-proj-v3/" # optional prefix added to each path ("" keeps as-is)
SAVE_DIR     = "/content/drive/MyDrive/mars_runs"  # where to save checkpoints/reports
IMG_SIZE     = 227
BATCH_SIZE   = 128
NUM_WORKERS  = 4                   # 2-4 is fine on Colab
PATIENCE     = 5                   # early stopping patience (epochs)
SEED         = 42

# Stage 1 (linear probe)
WARMUP_EPOCHS = 3                  # train only final layer
LR_FC         = 6e-3               # замораживаем голову, чекаем на что способен бекбон

# Stage 2 (full fine-tune)
FT_EPOCHS     = 50                 # max; early stopping will cut earlier
LR_FT         = 4e-4               # чуть меняем бекбон под задач
WEIGHT_DECAY  = 1e-4
LABEL_SMOOTH  = 0.05               # small smoothing helps noisy labels

os.makedirs(SAVE_DIR, exist_ok=True)

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.benchmark = True

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
# если не CUDA, то ИДИ И МЕНЯЙ НА T4 ИНАЧЕ УБЬЕЕЕЕТ!!!!!

Device: cuda


In [ ]:
# ––––––––––––––
# Functions
# ––––––––––––––
def parse_index(index_path: str) -> List[Tuple[str, int]]:
    items = []
    with open(index_path, newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            p = row["path"].strip()
            lab = int(row["label"]) if "label" in row else int(row["labels"])
            items.append((p, lab))
    if ROOT_DIR:
        items = [(os.path.join(ROOT_DIR, p), lab) for p, lab in items]
    return items

def subset(indices, data):
    return [data[i] for i in indices]

In [ ]:
# -----------------------
# Index parsing (TXT or CSV)
# -----------------------

raw_items = parse_index(INDEX_PATH)
print("Indexed:", len(raw_items))

# Map original numeric labels -> contiguous [0..K-1]
orig_labels = sorted({lab for _, lab in raw_items})
orig2idx = {lab:i for i, lab in enumerate(orig_labels)}
idx2orig = {i:lab for lab,i in orig2idx.items()}
data = [(p, orig2idx[lab]) for p, lab in raw_items]
classes = [str(ol) for ol in orig_labels]
num_classes = len(classes)
print("Num classes:", num_classes)
print("Original label → index mapping:", orig2idx)


Indexed: 21392
Num classes: 7
Original label → index mapping: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6}


In [ ]:
# -----------------------
# Train/Val/Test split (stratified 80/10/10)
# -----------------------
y = [lab for _, lab in data]
idx_all = np.arange(len(data))
idx_train, idx_test = train_test_split(idx_all, test_size=0.10, random_state=SEED, stratify=y)
y_train_tmp = [y[i] for i in idx_train]
idx_train, idx_val = train_test_split(idx_train, test_size=1/9, random_state=SEED, stratify=y_train_tmp)


train_items = subset(idx_train, data)
val_items   = subset(idx_val, data)
test_items  = subset(idx_test, data)
print(f"Splits -> train={len(train_items)}  val={len(val_items)}  test={len(test_items)}")


Splits -> train=17112  val=2140  test=2140


In [ ]:
# -----------------------
# Dataset / Transforms
# -----------------------
class MarsDataset(Dataset):
    """Single-label dataset; converts grayscale/rgba to RGB silently."""
    def __init__(self, items, transform):
        self.items = items
        self.transform = transform
    def __len__(self): return len(self.items)
    def __getitem__(self, i):
        p, lab = self.items[i]
        img = Image.open(p)
        if img.mode != "RGB":
            # unify to 3 channels
            if img.mode in ("L", "I;16"):
                img = img.convert("L")
                img = Image.merge("RGB", (img, img, img))
            else:
                img = img.convert("RGB")
        x = self.transform(img)
        return x, torch.tensor(lab, dtype=torch.long)

norm = transforms.Normalize(mean=[0.485,0.456,0.406],
                            std=[0.229,0.224,0.225])

base_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    norm,
])

train_ds = MarsDataset(train_items, base_tfms)
val_ds   = MarsDataset(val_items,   base_tfms)
test_ds  = MarsDataset(test_items,  base_tfms)

# Calculate weights for WeightedRandomSampler
y_train = [lab for _, lab in train_items]
counts = Counter(y_train)
w_class = {c: 1.0/max(counts[c],1) for c in range(num_classes)}
w_sample = [w_class[c] for c in y_train]
sampler = torch.utils.data.WeightedRandomSampler(w_sample, num_samples=len(w_sample), replacement=True)

train_loader = DataLoader(
    train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=NUM_WORKERS,
    pin_memory=True, persistent_workers=(NUM_WORKERS>0), prefetch_factor=2)
val_loader = DataLoader(
    val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS,
    pin_memory=True, persistent_workers=(NUM_WORKERS>0), prefetch_factor=2)
test_loader = DataLoader(
    test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS,
    pin_memory=True, persistent_workers=(NUM_WORKERS>0), prefetch_factor=2)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# -----------------------
# Model
# -----------------------
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)
model = model.to(device)

# Losses/optim
criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH)


optimizer_fc = torch.optim.AdamW(model.fc.parameters(), lr=LR_FC, weight_decay=WEIGHT_DECAY)
optimizer_ft = torch.optim.AdamW(model.parameters(),   lr=LR_FT, weight_decay=WEIGHT_DECAY)
scheduler_ft = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=FT_EPOCHS)

scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

def evaluate(loader):
    model.eval()
    total_loss = 0.0
    all_preds, all_targets = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
                logits = model(x)
                loss = criterion(logits, y) # Use the single criterion
            total_loss += loss.item() * x.size(0)
            preds = torch.argmax(logits, dim=1)
            all_preds.append(preds.detach().cpu().numpy())
            all_targets.append(y.detach().cpu().numpy())
    avg_loss = total_loss / len(loader.dataset)
    y_pred = np.concatenate(all_preds)
    y_true = np.concatenate(all_targets)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
    return avg_loss, f1, y_true, y_pred

best_f1 = -1.0
best_path = os.path.join(SAVE_DIR, "best.pt")
log_rows = []

# -----------------------
# Stage 1: Linear probe (fc only)
# -----------------------
for p in model.parameters():
    p.requires_grad = False
for p in model.fc.parameters():
    p.requires_grad = True

print("\n=== Stage 1: linear probe ===")
for epoch in range(1, WARMUP_EPOCHS+1):
    model.train()
    total_loss = 0.0
    n = 0
    for x, y in train_loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        optimizer_fc.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            logits = model(x)
            loss = criterion(logits, y) # Use the single criterion
        scaler.scale(loss).backward()
        scaler.step(optimizer_fc)
        scaler.update()
        total_loss += loss.item() * x.size(0)
        n += x.size(0)
    tr_loss = total_loss / n
    val_loss, val_f1, _, _ = evaluate(val_loader)
    log_rows.append(("stage1", epoch, tr_loss, val_loss, val_f1))
    print(f"[S1 {epoch:02d}] train_loss={tr_loss:.4f} | val_loss={val_loss:.4f} val_F1={val_f1:.4f}")
    # save if best
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save({"model": model.state_dict(),
                    "classes": classes,
                    "img_size": IMG_SIZE,
                    "idx2orig": idx2orig}, best_path)
        print(f"  → New best (F1={best_f1:.4f}) saved to {best_path}")

# -----------------------
# Stage 2: Full fine-tune
# -----------------------
for p in model.parameters():
    p.requires_grad = True

print("\n=== Stage 2: full fine-tune ===")
pat = PATIENCE
for epoch in range(1, FT_EPOCHS+1):
    model.train()
    total_loss = 0.0
    n = 0
    for x, y in train_loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        optimizer_ft.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            logits = model(x)
            loss = criterion(logits, y) # Use the single criterion
        scaler.scale(loss).backward()
        scaler.step(optimizer_ft)
        scaler.update()
        total_loss += loss.item() * x.size(0)
        n += x.size(0)
    scheduler_ft.step()
    tr_loss = total_loss / n
    val_loss, val_f1, _, _ = evaluate(val_loader)
    log_rows.append(("stage2", epoch, tr_loss, val_loss, val_f1))
    print(f"[S2 {epoch:02d}] train_loss={tr_loss:.4f} | val_loss={val_loss:.4f} val_F1={val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        pat = PATIENCE
        torch.save({"model": model.state_dict(),
                    "classes": classes,
                    "img_size": IMG_SIZE,
                    "idx2orig": idx2orig}, best_path)
        print(f"  → New best (F1={best_f1:.4f}) saved to {best_path}")
    else:
        pat -= 1
        if pat <= 0:
            print("  → Early stopping.")
            break

# Save training log
log_df = pd.DataFrame(log_rows, columns=["stage","epoch","train_loss","val_loss","val_f1"])
log_csv = os.path.join(SAVE_DIR, "train_log.csv")
log_df.to_csv(log_csv, index=False)
print("Log saved to:", log_csv)

# -----------------------
# Test evaluation (best checkpoint)
# -----------------------
ckpt = torch.load(best_path, map_location=device)
model.load_state_dict(ckpt["model"])
test_loss, test_f1, y_true, y_pred = evaluate(test_loader)
print(f"\nTEST: loss={test_loss:.4f}  macro-F1={test_f1:.4f}")
print(classification_report(y_true, y_pred, target_names=classes, digits=4))

cm = confusion_matrix(y_true, y_pred, labels=list(range(num_classes)))
cm_df = pd.DataFrame(cm, index=classes, columns=classes)
cm_csv = os.path.join(SAVE_DIR, "confusion_matrix.csv")
cm_df.to_csv(cm_csv)
print("Confusion matrix saved to:", cm_csv)

print("\nBest model:", best_path)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 188MB/s]



=== Stage 1: linear probe ===


/tmp/ipython-input-894261447.py:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))
/tmp/ipython-input-894261447.py:61: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
/tmp/ipython-input-894261447.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):


[S1 01] train_loss=0.6306 | val_loss=0.4818 val_F1=0.9251
  → New best (F1=0.9251) saved to /content/drive/MyDrive/mars_runs/best.pt


...

[S2 08] train_loss=0.2626 | val_loss=0.2709 val_F1=0.9964


/tmp/ipython-input-894261447.py:98: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
/tmp/ipython-input-894261447.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):


[S2 09] train_loss=0.2585 | val_loss=0.3191 val_F1=0.9683
  → Early stopping.
Log saved to: /content/drive/MyDrive/mars_runs/train_log.csv


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/tmp/ipython-input-894261447.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):



TEST: loss=0.2623  macro-F1=0.9976
              precision    recall  f1-score   support

           1     1.0000    0.9980    0.9990       490
           2     0.9912    0.9912    0.9912       228
           3     0.9957    0.9957    0.9957       466
           4     1.0000    1.0000    1.0000       350
           5     1.0000    1.0000    1.0000       185
           6     1.0000    1.0000    1.0000       230
           7     0.9948    1.0000    0.9974       191

    accuracy                         0.9977      2140
   macro avg     0.9974    0.9978    0.9976      2140
weighted avg     0.9977    0.9977    0.9977      2140

Confusion matrix saved to: /content/drive/MyDrive/mars_runs/confusion_matrix.csv

Best model: /content/drive/MyDrive/mars_runs/best.pt
